# Manual QC 2: Filter out subjects with bad template warps.
## Do this between bash scripts 5 and 6.
### This step is very important, as the final analysis mask is the intersection of all the template masks
Remove any bad subjects from the FBA derivatives directories before proceeding

In [ ]:
import nilearn
from nilearn.image import load_img
from nilearn.plotting import plot_img
import matplotlib.pyplot as plt
import glob
import numpy as np
import os

bids_dir = '/om4/group/gablab/data/hbn_bids/'
qsiprep_dir = bids_dir+'derivatives/qsiprep_fba_subs/'
fba_dir = bids_dir+'derivatives/fba/'
out_variable_dir = os.getcwd()+'/output_variables/'

In [ ]:
subs = [s.split('/')[-1] for s in glob.glob(fba_dir+'sub*')]
subs.sort()
# Load template FODs
fod_template = bids_dir+'/derivatives/fba/template/wmfod_template.nii.gz'
fod_img_full = load_img(fod_template)
fod_img = nilearn.image.index_img(fod_img_full, 0) # extract just the first slice for plotting purposes

# Make images 
index=0
num_images = np.ceil(np.size(subs)/50).astype(int)
# Create images with 50 brains in each, overlaying subject warped images on top template to check for good fit
for ite in range(num_images):
    print(ite)
    fig, axes = plt.subplots(10,5,figsize=(20, 20))
    for i in range(index,index+50):
        try:
            sub=subs[i]
            sub_img_path = fba_dir+sub+'/template/mask_template.nii.gz'
            img = load_img(sub_img_path)
            plot = plot_img(fod_img,axes=axes.flatten()[i-50*ite],title=sub,display_mode='x',cut_coords=[0])
            plot.add_overlay(img,cmap=nilearn.plotting.cm.purple_green,alpha=0.7)
        except Exception as e: print(e)
    index = index+50
    # Save image for future reference
    plt.savefig(out_variable_dir+'QC_iteration_'+str(ite))
    plt.show()